<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dcec5f5fd0181ae9b3220fd501c7f1770715aee7cc5999a9ea8f341acb9e1704
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 13:47:43
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -33.64 K (-0.24%)
Current PnL: -22.99 L (-15.06%)
CY Booked + Current PnL: -9.10 L (-5.96%)
-------------------
Total profit:  1.62 L
Total loss:  -24.61 L
-------------------
Total Booked + Current PnL: 17.88 L (14.24%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.73%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.40 L (61.9%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.06,-8.76,9.62,0.02,15321.0,-15293.0,159266.0,142.75,67.0,M-SC,11.73,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.42,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,37.0,M-SC,3.21,253.0,-0.99,0.59,10.32,OX40N,NTT,DURABLES
43,ITC,452.00,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.31,5.0,-0.10,1.39,4.36,X40,NTT,FMCG
83,VOLTAS,1530.00,-0.34,7.30,11.55,19.69,23763.0,13998.0,205740.0,-3.15,49.0,X-MC,3.24,78.0,0.59,1.44,14.59,XY25,NTT,AC
35,ICICIGI,2252.93,0.67,5.84,13.57,20.19,25036.0,10173.0,184493.0,-16.96,47.0,X-MC,7.14,68.0,0.41,1.29,21.83,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,4.89,-0.85,49.90,48.63,98931.0,-1696.0,198258.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.61,XR,NTT,MISC
81,VALIANTORG,838.0,4.68,-45.14,217.42,74.15,195156.0,-73845.0,89760.0,-261.57,37.0,H-SC,18.16,149.0,-0.38,0.63,15.06,XR,NTT,CHEMICALS
15,CAMS,4762.0,1.35,2.79,21.41,24.79,55426.0,7022.0,258878.0,-1.91,52.0,X-SC,1.89,86.0,0.13,1.81,26.57,X40N,NTT,MISC
76,TMPV,600.0,1.32,-14.34,65.98,42.18,156229.0,-39628.0,236782.0,-23.08,34.0,X-LC,2.32,3.0,-0.25,1.66,2.57,XY24,NTT,AUTO
44,JCHAC,2282.0,1.15,-28.46,39.76,-0.01,36355.0,-36368.0,91437.0,18617.39,22.0,M-SC,0.94,233.0,-1.00,0.64,3.54,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-6.34,-19.53,126.07,81.92,116460.0,-22421.0,92377.0,-52.84,20.0,M-SC,2.94,236.0,-0.19,0.65,9.36,XR,NTT,DURABLES
28,GREENPANEL,537.00,-2.30,-35.10,120.56,43.14,143241.0,-64265.0,118813.0,166.15,27.0,M-SC,6.34,240.0,-0.45,0.83,11.76,XY24,NTT,MISC
54,PAGEIND,51605.08,-2.21,-8.45,37.71,26.07,56527.0,-13840.0,149900.0,-32.53,28.0,X-MC,9.93,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
67,SIS,528.00,-2.02,-25.91,63.67,21.26,52377.0,-28769.0,82263.0,1938.34,41.0,H-SC,3.77,166.0,-0.55,0.58,11.15,OX40N,NTT,MISC
27,GLAXO,3466.20,-1.92,1.17,37.54,39.15,75687.0,2332.0,201616.0,-18.74,43.0,X-MC,10.52,60.0,0.03,1.41,30.51,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,1.07,0.32,106.06,106.72,131232.0,398.0,123734.0,-50.25,40.0,H-SC,2.41,139.0,0.00,0.87,23.78,AR,ATH,MISC
37,IEX,219.00,4.89,-0.85,49.90,48.63,98931.0,-1696.0,198258.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.61,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.32,-4.81,118.42,107.91,166427.0,-7104.0,140540.0,-22.58,37.0,M-SC,10.84,216.0,-0.04,0.98,0.49,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.06,-8.76,9.62,0.02,15321.0,-15293.0,159266.0,142.75,67.0,M-SC,11.73,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.42,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,37.0,M-SC,3.21,253.0,-0.99,0.59,10.32,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.61,-22.34,46.12,13.47,96581.0,-60255.0,209412.0,-68.12,34.0,H-SC,1.76,158.0,-0.62,1.47,6.48,XY24,NTT,PAINTS
67,SIS,528.00,-2.02,-25.91,63.67,21.26,52377.0,-28769.0,82263.0,1938.34,41.0,H-SC,3.77,166.0,-0.55,0.58,11.15,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.01,2.35,68.32,72.27,118483.0,3975.0,173423.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.67,XR,NTT,IT
38,INDIAMART,4810.62,1.07,0.32,106.06,106.72,131232.0,398.0,123734.0,-50.25,40.0,H-SC,2.41,139.0,0.00,0.87,23.78,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.01,2.35,68.32,72.27,118483.0,3975.0,173423.0,-11.98,59.0,M-LC,3.69,99.0,0.03,1.21,8.67,XR,NTT,IT
38,INDIAMART,4810.62,1.07,0.32,106.06,106.72,131232.0,398.0,123734.0,-50.25,40.0,H-SC,2.41,139.0,0.00,0.87,23.78,AR,ATH,MISC
37,IEX,219.00,4.89,-0.85,49.90,48.63,98931.0,-1696.0,198258.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.61,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.66,-1.27,37.37,35.63,77323.0,-2663.0,206911.0,-14.97,42.0,H-MC,3.70,119.0,-0.03,1.45,15.19,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.32,-4.81,118.42,107.91,166427.0,-7104.0,140540.0,-22.58,37.0,M-SC,10.84,216.0,-0.04,0.98,0.49,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.72,-36.65,113.44,35.20,92460.0,-47164.0,81506.0,6.86,20.0,X-SC,13.65,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-2.21,-8.45,37.71,26.07,56527.0,-13840.0,149900.0,-32.53,28.0,X-MC,9.93,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.27,-14.06,72.74,48.45,125342.0,-28191.0,172315.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.21,0.10,X40N,ATH,FINANCE
8,AWL,485.00,-1.10,-18.93,89.34,53.50,218546.0,-57128.0,244623.0,-60.69,34.0,X-MC,3.08,58.0,-0.26,1.71,8.65,XY24,NTT,FMCG
46,JSWINFRA,342.00,0.72,-8.26,25.60,15.23,47053.0,-16545.0,183802.0,-21.45,34.0,X-MC,6.88,66.0,-0.35,1.29,21.33,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.31,5.0,-0.10,1.39,4.36,X40,NTT,FMCG
3,ACC,3906.0,-0.03,-22.25,111.14,64.15,205609.0,-52951.0,185000.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.30,3.78,XY24,BTT,CEMENT
15,CAMS,4762.0,1.35,2.79,21.41,24.79,55426.0,7022.0,258878.0,-1.91,52.0,X-SC,1.89,86.0,0.13,1.81,26.57,X40N,NTT,MISC
66,SIEMENS,4671.5,0.15,-11.28,41.47,25.51,68467.0,-20995.0,165100.0,1.35,64.0,H-LC,2.03,49.0,-0.31,1.16,21.67,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.12,2.87,27.32,30.97,49669.0,5068.0,181806.0,-21.38,55.0,X-MC,2.12,75.0,0.10,1.27,15.79,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.72,-36.65,113.44,35.20,92460.0,-47164.0,81506.0,6.86,20.0,X-SC,13.65,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-2.21,-8.45,37.71,26.07,56527.0,-13840.0,149900.0,-32.53,28.0,X-MC,9.93,55.0,-0.24,1.05,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.27,-14.06,72.74,48.45,125342.0,-28191.0,172315.0,-20.30,31.0,X-MC,7.54,64.0,-0.22,1.21,0.10,X40N,ATH,FINANCE
55,PGHH,17907.65,0.21,-4.52,40.09,33.76,76871.0,-9075.0,191745.0,-32.64,34.0,X-MC,4.06,57.0,-0.12,1.34,0.44,X40,ATH,FMCG
19,COLPAL,3726.84,-0.67,-18.21,73.02,41.51,157285.0,-47965.0,215400.0,-2.56,40.0,X-MC,7.22,61.0,-0.30,1.51,1.08,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.62,-27.95,98.26,42.84,46020.0,-18171.0,46835.0,-52.62,41.0,X-SC,37.52,83.0,-0.39,0.33,1.91,XY24,NTT,MISC
60,RELAXO,1176.00,-0.12,-47.06,187.71,52.31,144246.0,-68315.0,76845.0,-43.66,37.0,X-SC,4.81,91.0,-0.47,0.54,2.86,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.72,-36.65,113.44,35.20,92460.0,-47164.0,81506.0,6.86,20.0,X-SC,13.65,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.15,-6.37,29.86,21.58,39180.0,-8928.0,131211.0,-52.80,35.0,X-MC,6.23,56.0,-0.23,0.92,16.16,X40N,ATH,IT
52,MEDANTA,1486.00,-0.51,3.50,19.88,24.08,26122.0,4448.0,131398.0,-4.85,46.0,X-SC,5.94,89.0,0.17,0.92,23.81,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.23,-14.94,40.24,19.28,118409.0,-51698.0,294258.0,-25.89,57.0,X-LC,7.02,1.0,-0.44,2.06,8.78,X40,ATH,IT
41,INFY,2275.00,-0.40,6.20,45.82,54.87,152260.0,19413.0,332301.0,-17.44,61.0,X-LC,2.48,2.0,0.13,2.33,15.00,X40,BTT,IT
76,TMPV,600.00,1.32,-14.34,65.98,42.18,156229.0,-39628.0,236782.0,-23.08,34.0,X-LC,2.32,3.0,-0.25,1.66,2.57,XY24,NTT,AUTO
82,VBL,671.64,0.30,-2.43,39.06,35.67,120365.0,-7688.0,308154.0,-14.05,63.0,X-LC,5.48,4.0,-0.06,2.16,11.08,X40N,ATH,FMCG
43,ITC,452.00,0.09,-1.14,11.72,10.44,23188.0,-2289.0,197849.0,-40.74,46.0,X-LC,1.31,5.0,-0.10,1.39,4.36,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.74,-14.40,112.24,81.67,90129.0,-13510.0,80300.0,7216.67,46.0,L-SC,19.36,271.0,-0.15,0.56,55.50,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.78,-12.16,58.86,39.54,52301.0,-12303.0,88856.0,-31.30,64.0,M-SC,6.68,220.0,-0.24,0.62,25.30,AR,ATH,AUTO
51,MASFIN,398.61,0.02,-5.43,29.06,22.05,26926.0,-5325.0,92655.0,-18.25,50.0,H-SC,6.80,164.0,-0.20,0.65,34.68,XR,ATH,FINANCE
70,SURYODAY,216.00,1.15,-18.59,51.12,23.03,74527.0,-33282.0,145789.0,57.89,51.0,H-SC,9.72,167.0,-0.45,1.02,44.65,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.06,-8.76,9.62,0.02,15321.0,-15293.0,159266.0,142.75,67.0,M-SC,11.73,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.06,-8.76,9.62,0.02,15321.0,-15293.0,159266.0,142.75,67.0,M-SC,11.73,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,-0.76,-0.30,21.31,20.95,53492.0,-748.0,251020.0,-1.34,69.0,X-LC,15.74,31.0,-0.01,1.76,35.23,X40,ATH,PAINTS
50,LTIM,7201.88,0.77,12.31,17.23,31.66,47634.0,30307.0,276458.0,8.03,68.0,H-LC,13.98,43.0,0.64,1.94,53.39,X200,ATH,IT
32,HCLTECH,1922.01,0.96,6.42,17.21,24.73,44307.0,15525.0,257449.0,11.06,65.0,X-LC,4.89,7.0,0.35,1.80,22.63,X40,ATH,IT
40,INDUSINDBK,1800.00,-1.41,-37.51,112.67,32.89,54358.0,-28962.0,48245.0,-695.74,59.0,L-MC,5.80,259.0,-0.53,0.34,32.86,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.97
1,25,44.86
2,50,75.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.46
MC    29.14
LC    25.47
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.73
X40      21.24
X40N     12.06
XR        9.61
XY25      9.10
AR        9.07
OX40N     7.59
X200      1.94
SR        0.98
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.66
X-LC    20.18
M-SC    11.86
X-SC     8.29
H-MC     4.76
H-LC     3.10
L-SC     1.40
M-MC     1.38
M-LC     1.21
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.86,-4.60,38.43
IT,13.85,-14.33,73.35
FINANCE,9.46,-14.85,63.38
MISC,7.41,-22.87,73.21
ELECTRICAL,6.01,-9.40,49.47
PAINTS,5.93,-9.21,25.43
INSURANCE,4.40,-0.44,35.84
PHARMA,3.90,-0.21,32.28
AUTO,3.39,-18.61,69.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3078225.0,21
XR,1290783.0,13
AR,1278952.0,10
X40,1003884.0,14
X40N,728864.0,9
OX40N,710067.0,10
XY25,343388.0,6
SR,280559.0,2
MH,77271.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3567637.0,25
M-SC,1366728.0,15
X-MC,1245441.0,16
X-LC,884326.0,11
X-SC,737475.0,8
H-MC,399264.0,3
L-SC,261829.0,3
M-LC,118483.0,1
H-LC,116101.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221665.0      6
           AR         879974.0      5
M-SC       XY24       787632.0      6
H-SC       XR         777985.0      7
X-MC       X40        443382.0      7
X-LC       X40        391656.0      5
X-MC       XY24       374646.0      3
H-SC       OX40N      330183.0      4
M-SC       OX40N      292277.0      5
X-SC       X40N       290878.0      3
H-SC       SR         280559.0      2
X-SC       XY24       277751.0      3
X-MC       X40N       246365.0      4
X-LC       XY24       226694.0      2
H-MC       AR         209427.0      2
X-LC       X40N       191621.0      2
H-MC       XY24       189837.0      1
X-MC       XY25       181048.0      2
L-SC       XR         174222.0      2
X-SC       X40        168846.0      2
M-SC       XR         165735.0      2
           AR         121084.0      2
M-LC       XR         118483.0      1
L-SC       OX40N       87607.0      1
H-SC       MH          77271.0      1
X-LC       XY25        74355.0      2
H-LC       AR          68467.0      1
L-MC       XR          54358.0      1
M-MC       XY25        48176.0      1
H-LC       X200        47634.0      1
L-LC       XY25        39809.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
